# CFA data extraction and data wrangling

In [1]:
import glob
import pandas as pd
from itertools import chain
import os

#set the output name of the csv files:
name = "A359_MSN216_S18LHS_SS"
folder_in = '01_Input/'
folder_out = '02_Data/'
folder_in = folder_in + name
folder_out = folder_out + name

paths = (r'\\de0-svm05\St_A350-1000_FAF_FDTTRANS\90_user\niklas\Deep_Learn\cfa\src_RS_A351TC_S18_SS', r'\\de0-svm05\St_A350-1000_FAF_FDTTRANS\90_user\niklas\Deep_Learn\cfa\src_RS_A359MSN21_S18_SS')
#parent_dir = r'/projects/St_A350-1000_FAF_FDTTRANS/78_python/03_LSI2RF/10_ML_CFA/01_Input'
parent_dir = r'/projects/St_A350-1000_FAF_FDTTRANS/81_cfa/src_UL_A359-216_S18_US/01_cfa'
parent_dir = folder_in

#subject_dirs = [os.path.join(parent_dir, dir) for dir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, dir))]
subject_dirs = [x[0] for x in sorted(os.walk(parent_dir))]
#subject_dirs = [x[0] for path in paths for x in sorted(os.walk(path))]
#my_paths = []
#for p in paths:
    #for path, dirs, files in os.walk(p):
        #my_paths.append(dirs)
# subject_dirs

In [2]:
#are both a and b node results inside one csv file?
ab_nodes = False
######## AC TYPE ###############
ac_types = ['A359', 'A351', 'A359-216']
ac_type = ac_types[2]

### Define relevant columns

In [3]:
list_loadA = ['AnalysisID','N_Fr_UL', 'M_Fr_UL', 'T_UL', 'f_x_Skin_L_UL', 'f_z_Skin_L_UL','f_xz_Skin_L_UL','f_x_Skin_R_UL', 
              'f_z_Skin_R_UL','f_xz_Skin_R_UL','Delta_p_UL','T_GFEM', 'Delta_Fx_UL']
list_result = ['A_Fr','I_Fr','EA_Fr','yCG_Fr','b_eff_111_UL','b_eff_121_UL','path']
list_Fmodes= ['RF_UL_FF_MS_comp','RF_UL_W_MS','RF_UL_FF_LS', 'RF_UL_JOINT_FR_to_CLIP_CL', 'RF_UL_JOINT_FR_to_CLIP_FR',
                'RF_UL_W_BUCK', 'RF_UL_FF_LB', 'RF_UL_JOINT_SK_to_CLIP_CL_R', 'RF_UL_JOINT_SK_to_CLIP_SK_R',
                   'sig_FF_UL', 'sig_OF_UL']
list_result = list_result + list_Fmodes
#list_result

## Import CFA data

In [4]:
i = 0
j = 0
filelist = []
for dir in subject_dirs:
    csv_files = [os.path.join(dir, csv) for csv in os.listdir(dir) if os.path.isfile(os.path.join(dir, csv)) and csv.endswith('.csv')]
    csv_files = sorted(csv_files)
    for file in csv_files:
        ##################################################################
        #if any(x in file for x in ['FE2SE','SE2FE']):
        if 'T.caesamexternalfiles' in file:
            curr_node = 'T'
        elif 'F.caesamexternalfiles' in file:
            curr_node = 'F'
        elif 'A.caesamexternalfiles' in file:
            curr_node = 'A'
        elif 'B.caesamexternalfiles' in file:
            curr_node = 'B'
        else:
            curr_node = 'UNDEFINED'
        ##################################################################
        if "LoadsA" in file:
            local_path = dir.split('/'[0])
            df1 = pd.read_csv(file, skiprows=2, index_col=1)
            i = i + 1
            filelist.append(file)
        if "ResultsPhaseA" in file:
            try:
                i == j + 1
            except ValueError:
                print ("No ResultsPhaseA File found at " , dir)
            else:
                df2 = pd.read_csv(file, skiprows=2, index_col=1)
                df2['path'] = local_path[-1]
                curr_loc1 = str(df2['AnalysisID'].tolist()[0].split('-')[-3]) #change this parameter!
                curr_loc2 = str(df2['AnalysisID'].tolist()[0].split('-')[-2])
                j = j + 1
                #this if clause considers that the A351 csv files contain both node A and B results
                if ab_nodes is False:
                    df3 = pd.merge(df1[list_loadA], 
                                   df2[list_result], 
                                   left_index=True, right_index=True, how='left')
                else:
                    df1 = df1[list_loadA].assign(key=df1.groupby(level=0).cumcount()).reset_index()
                    df2 = df2[list_result].assign(key=df2.groupby(level=0).cumcount()).reset_index()
                    df3 = pd.merge(df1,df2,how='left', on=['Load Case ID','key']).drop('key',1).set_index('Load Case ID')
            df3['Node'] = curr_node
            df3['Location_C'] = curr_loc1
            df3['Location_P'] = curr_loc2
            #df3['SEA'] = df3['AnalysisID'].str.replace(r'cfa_','')
            df3['SEA'] = df3['AnalysisID'].str.replace(r'FRA_C_','')
            if j == 1:
                #start
                #df3['Node'] = curr_node
                #df3['Location_C'] = curr_loc1
                #df3['Location_P'] = curr_loc2
                
                #end
                df_all = df3
            else:
                #start
                #df3['Node'] = curr_node
                #df3['Location_C'] = curr_loc1
                #df3['Location_P'] = curr_loc2
                #end
                df_all  = pd.concat([df_all, df3], ignore_index=False)
filelist

['01_Input/A359_MSN216_S18LHS_SS/CFA_OUT/GLA_MSN216_CID9_P28_C86_LHS_A.caesamexternalfiles/composite/frames/FRA_C_FRA-FR086-ST028-029/FRA_C_FRA-FR086-ST028-029_LoadsA_2017-05-11_12.28.17.csv',
 '01_Input/A359_MSN216_S18LHS_SS/CFA_OUT/GLA_MSN216_P15_LHS._B.caesamexternalfiles/composite/frames/FRA_C_FRA-FR072-ST015-016/FRA_C_FRA-FR072-ST015-016_LoadsA_2017-05-12_07.26.38.csv',
 '01_Input/A359_MSN216_S18LHS_SS/CFA_OUT/GLA_MSN216_P15_LHS._B.caesamexternalfiles/composite/frames/FRA_C_FRA-FR073-ST015-016/FRA_C_FRA-FR073-ST015-016_LoadsA_2017-05-12_07.27.42.csv',
 '01_Input/A359_MSN216_S18LHS_SS/CFA_OUT/GLA_MSN216_P15_LHS._B.caesamexternalfiles/composite/frames/FRA_C_FRA-FR074-ST015-016/FRA_C_FRA-FR074-ST015-016_LoadsA_2017-05-12_07.28.42.csv',
 '01_Input/A359_MSN216_S18LHS_SS/CFA_OUT/GLA_MSN216_P15_LHS._B.caesamexternalfiles/composite/frames/FRA_C_FRA-FR075-ST015-016/FRA_C_FRA-FR075-ST015-016_LoadsA_2017-05-12_07.29.43.csv',
 '01_Input/A359_MSN216_S18LHS_SS/CFA_OUT/GLA_MSN216_P15_LHS._B.caes

In [5]:
df_all = df_all.reset_index()
df_all = df_all.rename(columns={'Load Case ID': 'Load_Case_ID'})
df_all["GFEM_Label"] = df_all["Load_Case_ID"].str.split('_', 2).str[-1]
print(df_all.shape)
df_all.head()

(7888, 37)


,Load_Case_ID,AnalysisID,N_Fr_UL,M_Fr_UL,T_UL,f_x_Skin_L_UL,f_z_Skin_L_UL,f_xz_Skin_L_UL,f_x_Skin_R_UL,f_z_Skin_R_UL,...,RF_UL_FF_LB,RF_UL_JOINT_SK_to_CLIP_CL_R,RF_UL_JOINT_SK_to_CLIP_SK_R,sig_FF_UL,sig_OF_UL,Node,Location_C,Location_P,SEA,GFEM_Label
0,CLC_1_ZCT,FRA_C_FRA-FR086-ST028-029,94096.5,-96419.2,28562.6,208.847,111.0340,10.66520,233.027,84.4002,...,-9999.99,1.2346,1.5594,123.9500,114.6040,A,FR086,ST028,FRA-FR086-ST028-029,ZCT
1,CLC_2_DT1.15,FRA_C_FRA-FR086-ST028-029,53004.6,-54313.0,16089.3,117.643,62.5454,6.00771,131.264,47.5427,...,-9999.99,2.1917,2.7684,69.8209,64.5564,A,FR086,ST028,FRA-FR086-ST028-029,DT1.15
2,CLC_3_FATIGUE,FRA_C_FRA-FR086-ST028-029,42328.5,84910.2,16773.1,106.836,23.6275,24.64050,134.752,20.9564,...,-9999.99,2.1919,2.8619,49.8337,57.9871,A,FR086,ST028,FRA-FR086-ST028-029,FATIGUE
3,CLC_4_FATGL1,FRA_C_FRA-FR086-ST028-029,39742.4,65830.8,16393.4,105.836,42.2684,18.07720,128.138,35.0266,...,-9999.99,2.2432,2.9246,47.3978,53.7019,A,FR086,ST028,FRA-FR086-ST028-029,FATGL1
4,CLC_5_FATGL2,FRA_C_FRA-FR086-ST028-029,42821.9,83695.4,15967.3,104.630,11.0730,21.42870,129.691,10.5230,...,-9999.99,2.2789,2.9666,50.5152,58.5509,A,FR086,ST028,FRA-FR086-ST028-029,FATGL2


## Include information from LCDEF (moved to subsequent script)

In [6]:
'''if ac_type =='A359' and events == True:
    df5 = df_all
    df5["LoadCase"] = df5["Load_Case_ID"].str.split('_', 2).str[-1]
    #df_all["LoadCase"] = df_all.index.str.split("ALC_").str[1]
    #df5 = df5.set_index("LoadCase")
    dfLC = pd.read_csv('04_LCDEF/LCDEF_event.csv')
    dfLC = dfLC[['GFEM Label', 'Event', 'LC_type', 'Thermal', 'Pressure', 'LC_number']].dropna()
    dfLC['LC_number'] = dfLC['LC_number'].astype(int).astype(str)
    df6 = pd.merge(df5,dfLC,how='left',left_on=['LoadCase'], right_on=['GFEM Label'], sort=False).drop('GFEM Label', axis=1)
    
lcdef = ['ENV_ANT', 'FTVL_ANT', 'ENV', 'LS']    
curr_lcdef = lcdef[3]
    
if ac_type =='A351' and events == True:
    if curr_lcdef == 'ENV_ANT':
        df_lcdef1000 = pd.read_csv(r'\\de0-svm05\St_A350-1000_FAF_FDTTRANS\79_python_marco\LCDEF\LCDEF_A351-S18_ANT.csv')
        df_lcdef1000 = df_lcdef1000.drop(['Event', 'LC_type', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
    elif curr_lcdef == 'FTVL_ANT':
        df_lcdef1000 = pd.read_csv(r'\\de0-svm05\St_A350-1000_FAF_FDTTRANS\79_python_marco\LCDEF_A351-S18_FTVL_ANT.csv')
        df_lcdef1000 = df_lcdef1000.drop(['Event', 'LC_type', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
    elif curr_lcdef == 'ENV':
        df_lcdef1000 = pd.read_csv(r'\\de0-svm05\St_A350-1000_FAF_FDTTRANS\79_python_marco\LCDEF_A351-S18_ENV.csv')
        df_lcdef1000 = df_lcdef1000.drop(['Event', 'LC_type', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
    elif curr_lcdef == 'LS':
        df_lcdef1000 = pd.read_csv(r'\\de0-svm05\St_A350-1000_FAF_FDTTRANS\79_python_marco\LCDEF_A351-S18_FTVL_LS.csv')
        df_lcdef1000 = df_lcdef1000.drop(['Event', 'LC_type', 'Tmission', 'LC number'], axis=1)

    else:
        print('ERROR!!!')
    listx = df_lcdef1000['GFEM Label'].astype(str).tolist()
    listy = []
    for i in listx:
        if curr_lcdef == 'ENV_ANT':
            listy += ['ALC_' + i]
        elif curr_lcdef == 'FTVL_ANT':
            listy += ['ALC_' + i[4:]]
    df_lcdef1000['GFEM'] = pd.Series(listy)
#     df_lcdef1000 = df_lcdef1000.drop(['Event', 'LC_type', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
    df6 = pd.merge(df_all, df_lcdef1000, how='left', left_index=True, right_on=['GFEM Label']).drop('GFEM Label', axis=1)
    

df6.head()
'''

'if ac_type ==\'A359\' and events == True:\n    df5 = df_all\n    df5["LoadCase"] = df5["Load_Case_ID"].str.split(\'_\', 2).str[-1]\n    #df_all["LoadCase"] = df_all.index.str.split("ALC_").str[1]\n    #df5 = df5.set_index("LoadCase")\n    dfLC = pd.read_csv(\'04_LCDEF/LCDEF_event.csv\')\n    dfLC = dfLC[[\'GFEM Label\', \'Event\', \'LC_type\', \'Thermal\', \'Pressure\', \'LC_number\']].dropna()\n    dfLC[\'LC_number\'] = dfLC[\'LC_number\'].astype(int).astype(str)\n    df6 = pd.merge(df5,dfLC,how=\'left\',left_on=[\'LoadCase\'], right_on=[\'GFEM Label\'], sort=False).drop(\'GFEM Label\', axis=1)\n    \nlcdef = [\'ENV_ANT\', \'FTVL_ANT\', \'ENV\', \'LS\']    \ncurr_lcdef = lcdef[3]\n    \nif ac_type ==\'A351\' and events == True:\n    if curr_lcdef == \'ENV_ANT\':\n        df_lcdef1000 = pd.read_csv(r\'\\de0-svm05\\St_A350-1000_FAF_FDTTRANS\x079_python_marco\\LCDEF\\LCDEF_A351-S18_ANT.csv\')\n        df_lcdef1000 = df_lcdef1000.drop([\'Event\', \'LC_type\', \'Unnamed: 7\', \'Unnamed: 8

## Create standard position format

In [7]:
df7 = df_all
if ac_type =='A359':
    df7['Location_C2'] = df7['Location_C'].str.split('C').str[1]
    df7['Location_P2'] = df7['Location_P'].str.replace(r'Pr?','')
    df7.loc[df7['Location_P'].str.contains(r'^Pr|rTL'),'Location_side'] = 'RHS'
    df7.loc[df7['Location_P'].str.contains(r'^P\d|^TL'),'Location_side'] = 'LHS'
    #df7.loc[df7['Location_P']=='rTL','Location_P2'] = "0"
    df7.loc[df7['Location_P2'].str.contains(r'^rTL|^TL'),'Location_P2'] = '0'
    df7['Location'] = df7['Location_C2'] + "_" + df7['Location_P2'] + "_" + df7['Location_side']
elif ac_type == 'A359-216':
    df7['Location_C2'] = df7['Location_C'].str.split('FR0').str[1]
    df7['Location_P2'] = df7['Location_P'].str.split('ST').str[1].str.replace(r'^0+','')
    df7['Location_side'] = 'LHS'
    df7['Location'] = df7['Location_C2'] + "_" + df7['Location_P2'] + "_" + df7['Location_side']
    
else:
    df7['Location_C2'] = df7['Location_C'].str.split('FR0').str[1]
    df7['Location_P2'] = df7['Location_P'].str.split('STR').str[1].str.slice(stop=3).str.replace(r'^0+','')
    df7['Location_side'] = df7['path'].str.split('_').str[-4] +"HS"
    df7.loc[df7['Location_P']=='STR001-STR001','Location_P2'] = "0"
    df7['Location'] = df7['Location_C2'] + "_" + df7['Location_P2'] + "_" + df7['Location_side']
#df7.head(20)

## create list of relevant elements 

In [8]:
input_qvlv = '05_QVLV/QVLV.csv'
df_qvlv = pd.read_csv(input_qvlv)
df_qvlv.head()

,QorL,ISSY,Ele,PosCP
0,QV,20C20,20,C20
1,QV,21C21,21,C21
2,QV,22C22,22,C22
3,QV,23C22A,23,C22A
4,QV,24C23,24,C23


In [9]:
df_qv = df_qvlv.loc[df_qvlv['QorL']=='QV',['PosCP', 'Ele']]
df_qv['Location_C'] = df_qv['PosCP'].str.replace(r'C','')
df_lv = df_qvlv.loc[df_qvlv['QorL']=='LV',['PosCP', 'Ele']]
df_lv.loc[df_lv['PosCP'].str.contains(r'^P\''),'Location_side'] = 'RHS'
df_lv.loc[df_lv['PosCP'].str.contains(r'^P\d'),'Location_side'] = 'LHS'
df_lv.loc[df_lv['PosCP']=='TL','Location_side'] = 'LHS'
df_lv.loc[df_lv['PosCP']=='*TL','Location_side'] = 'RHS'
df_lv['Location_P'] = df_lv['PosCP'].str.replace(r'P\'?','')
df_lv.loc[df_lv['PosCP']=='*TL','Location_P'] = 'TL'
df_lv.loc[df_lv['Location_P'].str.contains(r'^rTL|^TL'),'Location_P'] = '0'
df_lv['P_side'] = df_lv['Location_P'] + '_' + df_lv['Location_side']

#df_qv.loc[df_qv['PosCP']=='C20','Ele']

In [10]:
df7['Location'].unique()

array(['86_28_LHS', '72_15_LHS', '73_15_LHS', '74_15_LHS', '75_15_LHS',
       '76_15_LHS', '77_15_LHS', '78_15_LHS', '79_15_LHS', '80_15_LHS',
       '81_15_LHS', '82_15_LHS', '83_15_LHS', '84_15_LHS', '85_15_LHS',
       '91_15_LHS', '92_15_LHS'], dtype=object)

In [11]:
df_el = pd.DataFrame()
df_el['Location'] = df7['Location'].unique()
df_el[['Location_C', 'Location_P', 'Location_side']] = df_el['Location'].str.split('_',expand=True)
df_el['Location_C-1'] = (df_el['Location_C'].astype(int) - 1).astype(str)
df_el['P_side'] = df_el['Location_P'] + '_' + df_el['Location_side']
df_el.head()

,Location,Location_C,Location_P,Location_side,Location_C-1,P_side
0,86_28_LHS,86,28,LHS,85,28_LHS
1,72_15_LHS,72,15,LHS,71,15_LHS
2,73_15_LHS,73,15,LHS,72,15_LHS
3,74_15_LHS,74,15,LHS,73,15_LHS
4,75_15_LHS,75,15,LHS,74,15_LHS


In [12]:
def p_format(p_number):
    p_str = str(p_number)
    if len(p_str) == 1:
        return  '00' + p_str
    elif len(p_str) == 2:
        return  '0' + p_str
    else:
        return p_str
df_el2 = pd.merge(df_el, df_lv[['P_side','Ele']], on='P_side',how='left')
df_el2 = df_el2.rename(columns={'Ele': 'EleP'})
df_el2['EleP'] = df_el2['EleP'].astype(str)
df_el2['EleP2'] = df_el2['EleP'].apply(p_format)
df_el3 = pd.merge(df_el2, df_qv[['Location_C','Ele']], on='Location_C',how='left')
df_el3 = df_el3.rename(columns={'Ele': 'EleC'})
df_el4 = pd.merge(df_el3, df_qv[['Location_C','Ele']], left_on=['Location_C-1'], right_on=['Location_C'], how='left')
df_el4 = df_el4.rename(columns={'Ele': 'EleC-1'})
df_el4 =df_el4.drop('Location_C_y', axis =1)
df_el4['FrameEl'] = df_el4['EleC'].astype(str) + df_el4['EleP2'] + "10"
df_el4['SkinEl_LHS'] = df_el4['EleC-1'].astype(str) + df_el4['EleP2'] + "30"
df_el4['SkinEl_RHS'] = df_el4['EleC'].astype(str) + df_el4['EleP2'] + "30"

#df_el4.head()

## remove portion of the data, write out csv

In [13]:
#df2 = df_all.iloc[::10, :]
#df2 = df5[df5.index % 20 == 0]
if not os.path.exists(folder_out):
    os.mkdir(folder)
df7.to_pickle(folder_out+'/' +'df_'+name + '_cfa.pkl')

if not os.path.exists(folder_out+ '/QVLV'):
    os.mkdir(folder_out+ '/QVLV')
df_el4.to_csv(folder_out+'/QVLV/'+ name + '_Ele.csv')


In [14]:
#df.to_hdf(folder+'/'+name_out + '.h5','RF_table',mode='w',format='table',data_columns=True)

In [15]:
df7.head()

,Load_Case_ID,AnalysisID,N_Fr_UL,M_Fr_UL,T_UL,f_x_Skin_L_UL,f_z_Skin_L_UL,f_xz_Skin_L_UL,f_x_Skin_R_UL,f_z_Skin_R_UL,...,sig_OF_UL,Node,Location_C,Location_P,SEA,GFEM_Label,Location_C2,Location_P2,Location_side,Location
0,CLC_1_ZCT,FRA_C_FRA-FR086-ST028-029,94096.5,-96419.2,28562.6,208.847,111.0340,10.66520,233.027,84.4002,...,114.6040,A,FR086,ST028,FRA-FR086-ST028-029,ZCT,86,28,LHS,86_28_LHS
1,CLC_2_DT1.15,FRA_C_FRA-FR086-ST028-029,53004.6,-54313.0,16089.3,117.643,62.5454,6.00771,131.264,47.5427,...,64.5564,A,FR086,ST028,FRA-FR086-ST028-029,DT1.15,86,28,LHS,86_28_LHS
2,CLC_3_FATIGUE,FRA_C_FRA-FR086-ST028-029,42328.5,84910.2,16773.1,106.836,23.6275,24.64050,134.752,20.9564,...,57.9871,A,FR086,ST028,FRA-FR086-ST028-029,FATIGUE,86,28,LHS,86_28_LHS
3,CLC_4_FATGL1,FRA_C_FRA-FR086-ST028-029,39742.4,65830.8,16393.4,105.836,42.2684,18.07720,128.138,35.0266,...,53.7019,A,FR086,ST028,FRA-FR086-ST028-029,FATGL1,86,28,LHS,86_28_LHS
4,CLC_5_FATGL2,FRA_C_FRA-FR086-ST028-029,42821.9,83695.4,15967.3,104.630,11.0730,21.42870,129.691,10.5230,...,58.5509,A,FR086,ST028,FRA-FR086-ST028-029,FATGL2,86,28,LHS,86_28_LHS
